##  We're now going to read the turnout data and analyze turnout by precinct for only 2012

In [1]:
#First set everthing up
import pyodbc
import graphlab as gl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import local #server information and credentials
from pullODBC import pullODBC
from pullODBCall import pullODBCall
from matplotlib.backends.backend_pdf import PdfPages
from __future__ import division # convert integer division to floating point
%matplotlib inline
plt.style.use('seaborn-white')
plt.rcParams['font.size'] = 7
plt.tight_layout;
plt.rc('xtick', labelsize=7) 
plt.rc('ytick', labelsize=7) 

### Section A: Analyzing Turnout by Precinct for Voters Affiliated with a Party

Read in the csv for the combined data: county, precinct, party code, and 2106 turnout

In [2]:
combined = pd.read_csv('C:/voter/combined2012.csv', low_memory=True)

C:\Users\Scott\Anaconda2\envs\gl-env\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print len(combined)

6649071


In [4]:
print combined['2012'].sum()

4520555.0


Print out the columns of this dataframe

In [5]:
print combined.columns

Index([u'Unnamed: 0', u'birth_age', u'gender_code', u'race_code',
       u'ethnic_code', u'party_cd', u'status_cd', u'zip_code', u'birth_state',
       u'county_desc', u'registr_dt', u'ncid', u'precinct_abbrv',
       u'Unnamed: 0.1', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012',
       u'2013', u'2014', u'2015', u'2016', u'2018'],
      dtype='object')


In [6]:
print combined.head()

   Unnamed: 0  birth_age gender_code race_code ethnic_code party_cd status_cd  \
0           0       84.0           F       W           NL       DEM        A    
1           1       43.0           M       W           NL       REP        A    
2           2       51.0           F       W           NL       REP        A    
3           3       57.0           M       W           NL       UNA        A    
4           4       46.0           F       W           NL       REP        A    

  zip_code                     birth_state county_desc  ...  2008 2009 2010  \
0    27215  NC                                ALAMANCE  ...   NaN  NaN  NaN   
1    27258  NC                                ALAMANCE  ...   1.0  0.0  1.0   
2    27258  TN                                ALAMANCE  ...   1.0  0.0  0.0   
3    27217  NC                                ALAMANCE  ...   NaN  NaN  NaN   
4    27215  NC                                ALAMANCE  ...   1.0  0.0  1.0   

   2011  2012  2013  2014  2015  2016 

Subset the dataframe to leave only the relevent columns 

In [7]:
turnout = combined.ix[:,['county_desc', 'party_cd', 'precinct_abbrv', '2012']]
del combined

In [8]:
print turnout.head()

  county_desc party_cd precinct_abbrv  2012
0    ALAMANCE      DEM         124      NaN
1    ALAMANCE      REP         13       1.0
2    ALAMANCE      REP         09N      1.0
3    ALAMANCE      UNA         04       NaN
4    ALAMANCE      REP         02       1.0


Creat a custom aggfunc to count the percentage of rows with '1' in column '2016'

In [9]:
def percent_non_zero(x):
    frac = np.sum(x)/np.size(x)
    return(frac)

Create a Pivot Table with index of county, precinct and columns party_cd and aggregation 2016

In [10]:
registered_by_precinct = pd.pivot_table(turnout, index=['county_desc','precinct_abbrv'], values=['2012'],  
                                 aggfunc=percent_non_zero, columns=['party_cd'], margins = False) 


In [15]:
pd.options.display.float_format = '{:.1%}'.format #display as percentages
new_dataframe = pd.DataFrame.from_records(registered_by_precinct.to_records())
new_dataframe = new_dataframe.iloc[1:]
print new_dataframe.head()




  county_desc precinct_abbrv  ('2012', 'DEM')  ('2012', 'LIB')  \
1    ALAMANCE         01                68.3%            90.0%   
2    ALAMANCE         02                72.8%            62.5%   
3    ALAMANCE         035               60.1%            44.4%   
4    ALAMANCE         03C               78.6%           100.0%   
5    ALAMANCE         03N               69.5%            66.7%   

   ('2012', 'REP')  ('2012', 'UNA')  
1            77.0%            59.7%  
2            81.3%            66.2%  
3            69.4%            49.5%  
4            84.2%            71.3%  
5            76.8%            60.3%  


In [16]:
new_dataframe.to_csv('C:/voter/turnout2012.csv')

In [17]:
print new_dataframe.head()

  county_desc precinct_abbrv  ('2012', 'DEM')  ('2012', 'LIB')  \
1    ALAMANCE         01                68.3%            90.0%   
2    ALAMANCE         02                72.8%            62.5%   
3    ALAMANCE         035               60.1%            44.4%   
4    ALAMANCE         03C               78.6%           100.0%   
5    ALAMANCE         03N               69.5%            66.7%   

   ('2012', 'REP')  ('2012', 'UNA')  
1            77.0%            59.7%  
2            81.3%            66.2%  
3            69.4%            49.5%  
4            84.2%            71.3%  
5            76.8%            60.3%  
